In [ ]:
!pip3 install google-cloud-aiplatform --user
!pip3 install pyarrow==11.0.0 --user
!pip3 install --upgrade google-cloud-bigquery --user
!pip3 install --upgrade google-cloud-bigquery-storage --user
!pip3 install --upgrade google-cloud-storage --user
!pip install db-dtypes

### Define constants & import libraries

In [1]:
PROJECT_ID = 'york-bb-cohort'
BQ_LOCATION = 'US'
REGION = 'us-east1'
GCS_BUCKET = f"{PROJECT_ID}-akane_bucket"
# Bucket created via console

In [2]:
from google.cloud import bigquery
from google.cloud import aiplatform as vertexai
import numpy as np
import pandas as pd

In [3]:
BQ_DATASET = f"{PROJECT_ID}:akane_bqds"

In [7]:
!bq mk --location={BQ_LOCATION} --dataset {BQ_DATASET}

Dataset 'york-bb-cohort:akane_bqds' successfully created.


### Initialize Vertex Python SDK client

In [4]:
vertexai.init(project=PROJECT_ID, location=REGION, staging_bucket=f"gs://{GCS_BUCKET}")

### Preview some data

In [5]:
from google.cloud.bigquery import Client, QueryJobConfig
client = Client()

In [7]:
# Total # of hits and target occurrences for March 2017
%%bigquery --project $PROJECT_ID
SELECT
    COUNT(fullVisitorId) as session_count,
    SUM(totals.hits) as total_hits
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_201703*`

Query is running:   0%|          |

Downloading:   0%|          |

,session_count,total_hits
0,69931,300909


In [8]:
%%bigquery --project $PROJECT_ID
SELECT
    COUNT(hits.eCommerceAction.action_type) AS add_cart_count
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_201703*`,
UNNEST(hits) AS hits
WHERE hits.eCommerceAction.action_type = '3'

Query is running:   0%|          |

Downloading:   0%|          |

,add_cart_count
0,8782


In [13]:
# Look at 2% of March 2017 data

query = """SELECT * FROM `bigquery-public-data.google_analytics_sample.ga_sessions_201703*` 
TABLESAMPLE SYSTEM (2 PERCENT)"""
job = client.query(query)
df = job.to_dataframe()

In [14]:
df.head()

,visitorId,visitNumber,visitId,visitStartTime,date,totals,trafficSource,device,geoNetwork,customDimensions,hits,fullVisitorId,userId,channelGrouping,socialEngagementType
0,<NA>,1,1489509160,1489509160,20170314,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'North...",[],"[{'hitNumber': 1, 'time': 0, 'hour': 9, 'minut...",3909627023745010729,None,Organic Search,Not Socially Engaged
1,<NA>,1,1489493877,1489493877,20170314,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': '(not set)'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'Weste...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 5, 'minut...",5973194633724231760,None,Organic Search,Not Socially Engaged
2,<NA>,2,1489484027,1489484027,20170314,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': None, 'campaign': 'Data Share...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'Weste...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 2, 'minut...",8698935020465113969,None,Affiliates,Not Socially Engaged
3,<NA>,2,1489530179,1489530179,20170314,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...","{'referralPath': '/analytics/web/', 'campaign'...","{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'South...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 15, 'minu...",5901196461048284123,None,Referral,Not Socially Engaged
4,<NA>,17,1489480040,1489480040,20170314,"{'visits': 1, 'hits': 1, 'pageviews': 1, 'time...",{'referralPath': '/analytics/web/inpage_launch...,"{'browser': 'Chrome', 'browserVersion': 'not a...","{'continent': 'Europe', 'subContinent': 'Weste...","[{'index': 4, 'value': 'EMEA'}]","[{'hitNumber': 1, 'time': 0, 'hour': 1, 'minut...",0209659091905351565,None,Referral,Not Socially Engaged


Need strategy to deal with nested data before cleaning.

In [16]:
df.iloc[0]

visitorId                                                            <NA>
visitNumber                                                             1
visitId                                                        1489509160
visitStartTime                                                 1489509160
date                                                             20170314
totals                  {'visits': 1, 'hits': 1, 'pageviews': 1, 'time...
trafficSource           {'referralPath': None, 'campaign': '(not set)'...
device                  {'browser': 'Chrome', 'browserVersion': 'not a...
geoNetwork              {'continent': 'Europe', 'subContinent': 'North...
customDimensions                                                       []
hits                    [{'hitNumber': 1, 'time': 0, 'hour': 9, 'minut...
fullVisitorId                                         3909627023745010729
userId                                                               None
channelGrouping                       

### Test set will be made out of raw data for March 2017.
#### This query retrieves all records for that month with all nested records flattened and creates a DataFrame.

In [6]:
query = """SELECT
visitorId,
visitNumber,
visitId,
visitStartTime,
date,
fullVisitorId,
userId,
channelGrouping,
socialEngagementType,
h.hitNumber,
h.time,
h.hour,
h.minute,
h.isSecure,
h.isInteraction,
h.isEntrance,
h.isExit,
h.referer,
h.type,
h.dataSource,
h.page.pagePath,
h.page.hostname,
h.page.pageTitle,
h.page.searchKeyword,
h.page.searchCategory,
h.page.pagePathLevel1,
h.page.pagePathLevel2,
h.page.pagePathLevel3,
h.page.pagePathLevel4,
h.transaction.transactionId,
h.transaction.transactionRevenue,
h.transaction.transactionTax,
h.transaction.transactionShipping,
h.transaction.affiliation,
h.transaction.currencyCode,
h.transaction.localTransactionRevenue,
h.transaction.localTransactionTax,
h.transaction.localTransactionShipping,
h.transaction.transactionCoupon,
h.item.transactionId,
h.item.productName,
h.item.productCategory,
h.item.productSku,
h.item.itemQuantity,
h.item.itemRevenue,
h.item.currencyCode,
h.item.localItemRevenue,
h.contentInfo.contentDescription,
h.appInfo.name,
h.appInfo.version,
h.appInfo.id,
h.appInfo.installerId,
h.appInfo.appInstallerId,
h.appInfo.appName,
h.appInfo.appVersion,
h.appInfo.appId,
h.appInfo.screenName,
h.appInfo.landingScreenName,
h.appInfo.exitScreenName,
h.appInfo.screenDepth,
h.exceptionInfo.description,
h.exceptionInfo.isFatal,
h.exceptionInfo.exceptions,
h.exceptionInfo.fatalExceptions,
h.eventInfo.eventCategory,
h.eventInfo.eventAction,
h.eventInfo.eventLabel,
h.eventInfo.eventValue,
hp.productSKU,
hp.v2ProductName,
hp.v2ProductCategory,
hp.productVariant,
hp.productBrand,
hp.productRevenue,
hp.localProductRevenue,
hp.productPrice,
hp.localProductPrice,
hp.productQuantity,
hp.productRefundAmount,
hp.localProductRefundAmount,
hp.isImpression,
hp.isClick,
hpcd.index,
hpcd.value,
hpcm.index,
hpcm.value,
hp.productListName,
hp.productListPosition,
hpr.promoId,
hpr.promoName,
hpr.promoCreative,
hpr.promoPosition,
h.promotionActionInfo.promoIsView,
h.promotionActionInfo.promoIsClick,
h.refund.refundAmount,
h.refund.localRefundAmount,
h.eCommerceAction.action_type,
h.eCommerceAction.step,
h.eCommerceAction.option,
hex.experimentId,
hex.experimentVariant,
h.publisher.dfpClicks,
h.publisher.dfpImpressions,
h.publisher.dfpMatchedQueries,
h.publisher.dfpMeasurableImpressions,
h.publisher.dfpQueries,
h.publisher.dfpRevenueCpm,
h.publisher.dfpRevenueCpc,
h.publisher.dfpViewableImpressions,
h.publisher.dfpPagesViewed,
h.publisher.adsenseBackfillDfpClicks,
h.publisher.adsenseBackfillDfpImpressions,
h.publisher.adsenseBackfillDfpMatchedQueries,
h.publisher.adsenseBackfillDfpMeasurableImpressions,
h.publisher.adsenseBackfillDfpQueries,
h.publisher.adsenseBackfillDfpRevenueCpm,
h.publisher.adsenseBackfillDfpRevenueCpc,
h.publisher.adsenseBackfillDfpViewableImpressions,
h.publisher.adsenseBackfillDfpPagesViewed,
h.publisher.adxBackfillDfpClicks,
h.publisher.adxBackfillDfpImpressions,
h.publisher.adxBackfillDfpMatchedQueries,
h.publisher.adxBackfillDfpMeasurableImpressions,
h.publisher.adxBackfillDfpQueries,
h.publisher.adxBackfillDfpRevenueCpm,
h.publisher.adxBackfillDfpRevenueCpc,
h.publisher.adxBackfillDfpViewableImpressions,
h.publisher.adxBackfillDfpPagesViewed,
h.publisher.adxClicks,
h.publisher.adxImpressions,
h.publisher.adxMatchedQueries,
h.publisher.adxMeasurableImpressions,
h.publisher.adxQueries,
h.publisher.adxRevenue,
h.publisher.adxViewableImpressions,
h.publisher.adxPagesViewed,
h.publisher.adsViewed,
h.publisher.adsUnitsViewed,
h.publisher.adsUnitsMatched,
h.publisher.viewableAdsViewed,
h.publisher.measurableAdsViewed,
h.publisher.adsPagesViewed,
h.publisher.adsClicked,
h.publisher.adsRevenue,
h.publisher.dfpAdGroup,
h.publisher.dfpAdUnits,
h.publisher.dfpNetworkId,
hcv.index,
hcv.customVarName,
hcv.customVarValue,
hcd.index,
hcd.value,
hcm.index,
hcm.value,
h.social.socialInteractionNetwork,
h.social.socialInteractionAction,
h.social.socialInteractions,
h.social.socialInteractionTarget,
h.social.socialNetwork,
h.social.uniqueSocialInteractions,
h.social.hasSocialSourceReferral,
h.social.socialInteractionNetworkAction,
h.latencyTracking.pageLoadSample,
h.latencyTracking.pageLoadTime,
h.latencyTracking.pageDownloadTime,
h.latencyTracking.redirectionTime,
h.latencyTracking.speedMetricsSample,
h.latencyTracking.domainLookupTime,
h.latencyTracking.serverConnectionTime,
h.latencyTracking.serverResponseTime,
h.latencyTracking.domLatencyMetricsSample,
h.latencyTracking.domInteractiveTime,
h.latencyTracking.domContentLoadedTime,
h.latencyTracking.userTimingValue,
h.latencyTracking.userTimingSample,
h.latencyTracking.userTimingVariable,
h.latencyTracking.userTimingCategory,
h.latencyTracking.userTimingLabel,
h.sourcePropertyInfo.sourcePropertyDisplayName,
h.sourcePropertyInfo.sourcePropertyTrackingId,
h.contentGroup.contentGroup1,
h.contentGroup.contentGroup2,
h.contentGroup.contentGroup3,
h.contentGroup.contentGroup4,
h.contentGroup.contentGroup5,
h.contentGroup.previousContentGroup1,
h.contentGroup.previousContentGroup2,
h.contentGroup.previousContentGroup3,
h.contentGroup.previousContentGroup4,
h.contentGroup.previousContentGroup5,
h.contentGroup.contentGroupUniqueViews1,
h.contentGroup.contentGroupUniqueViews2,
h.contentGroup.contentGroupUniqueViews3,
h.contentGroup.contentGroupUniqueViews4,
h.contentGroup.contentGroupUniqueViews5,
hpi.dfpClicks,
hpi.dfpImpressions,
hpi.dfpMatchedQueries,
hpi.dfpMeasurableImpressions,
hpi.dfpQueries,
hpi.dfpRevenueCpm,
hpi.dfpRevenueCpc,
hpi.dfpViewableImpressions,
hpi.dfpPagesViewed,
hpi.adsenseBackfillDfpClicks,
hpi.adsenseBackfillDfpImpressions,
hpi.adsenseBackfillDfpMatchedQueries,
hpi.adsenseBackfillDfpMeasurableImpressions,
hpi.adsenseBackfillDfpQueries,
hpi.adsenseBackfillDfpRevenueCpm,
hpi.adsenseBackfillDfpRevenueCpc,
hpi.adsenseBackfillDfpViewableImpressions,
hpi.adsenseBackfillDfpPagesViewed,
hpi.adxBackfillDfpClicks,
hpi.adxBackfillDfpImpressions,
hpi.adxBackfillDfpMatchedQueries,
hpi.adxBackfillDfpMeasurableImpressions,
hpi.adxBackfillDfpQueries,
hpi.adxBackfillDfpRevenueCpm,
hpi.adxBackfillDfpRevenueCpc,
hpi.adxBackfillDfpViewableImpressions,
hpi.adxBackfillDfpPagesViewed,
hpi.adxClicks,
hpi.adxImpressions,
hpi.adxMatchedQueries,
hpi.adxMeasurableImpressions,
hpi.adxQueries,
hpi.adxRevenue,
hpi.adxViewableImpressions,
hpi.adxPagesViewed,
hpi.adsViewed,
hpi.adsUnitsViewed,
hpi.adsUnitsMatched,
hpi.viewableAdsViewed,
hpi.measurableAdsViewed,
hpi.adsPagesViewed,
hpi.adsClicked,
hpi.adsRevenue,
hpi.dfpAdGroup,
hpi.dfpAdUnits,
hpi.dfpNetworkId,
trafficSource.referralPath,
trafficSource.campaign,
trafficSource.source,
trafficSource.medium,
trafficSource.keyword,
trafficSource.adContent,
trafficSource.isTrueDirect,
trafficSource.campaignCode,
trafficSource.adwordsClickInfo.campaignId,
trafficSource.adwordsClickInfo.adGroupId,
trafficSource.adwordsClickInfo.creativeId,
trafficSource.adwordsClickInfo.criteriaId,
trafficSource.adwordsClickInfo.page,
trafficSource.adwordsClickInfo.slot,
trafficSource.adwordsClickInfo.criteriaParameters,
trafficSource.adwordsClickInfo.gclId,
trafficSource.adwordsClickInfo.customerId,
trafficSource.adwordsClickInfo.adNetworkType,
trafficSource.adwordsClickInfo.isVideoAd,
trafficSource.adwordsClickInfo.targetingCriteria.boomUserlistId,
totals.visits,
totals.hits,
totals.pageviews,
totals.timeOnSite,
totals.bounces,
totals.transactions,
totals.transactionRevenue,
totals.newVisits,
totals.screenviews,
totals.uniqueScreenviews,
totals.timeOnScreen,
totals.totalTransactionRevenue,
totals.sessionQualityDim,
device.browser,
device.browserVersion,
device.browserSize,
device.operatingSystem,
device.operatingSystemVersion,
device.isMobile,
device.mobileDeviceBranding,
device.mobileDeviceModel,
device.mobileInputSelector,
device.mobileDeviceInfo,
device.mobileDeviceMarketingName,
device.flashVersion,
device.javaEnabled,
device.language,
device.screenColors,
device.screenResolution,
device.deviceCategory,
geoNetwork.continent,
geoNetwork.subContinent,
geoNetwork.country,
geoNetwork.region,
geoNetwork.metro,
geoNetwork.city,
geoNetwork.cityId,
geoNetwork.networkDomain,
geoNetwork.latitude,
geoNetwork.longitude,
geoNetwork.networkLocation
FROM `bigquery-public-data.google_analytics_sample.ga_sessions_201703*`
left join unnest(hits) as h
left join unnest(h.product) as hp
left join unnest(hp.customDimensions) as hpcd
left join unnest(hp.customMetrics) as hpcm
left join unnest(h.promotion) as hpr
left join unnest(h.experiment) as hex
left join unnest(h.customVariables) as hcv
left join unnest(h.customDimensions) as hcd
left join unnest(h.customMetrics) as hcm
left join unnest(h.publisher_infos) as hpi"""
job = client.query(query)
df_raw_flat = job.to_dataframe()

In [7]:
df_raw_flat.head()

,visitorId,visitNumber,visitId,visitStartTime,date,fullVisitorId,userId,channelGrouping,socialEngagementType,hitNumber,...,subContinent,country,region,metro,city,cityId,networkDomain,latitude,longitude,networkLocation
0,<NA>,1,1489529400,1489529400,20170314,2838753039303821930,None,Referral,Not Socially Engaged,16,...,Northern Europe,Norway,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,lyse.net,not available in demo dataset,not available in demo dataset,not available in demo dataset
1,<NA>,1,1489529400,1489529400,20170314,2838753039303821930,None,Referral,Not Socially Engaged,44,...,Northern Europe,Norway,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,lyse.net,not available in demo dataset,not available in demo dataset,not available in demo dataset
2,<NA>,7,1489550131,1489550131,20170314,119870259714905967,None,Direct,Not Socially Engaged,20,...,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset
3,<NA>,1,1489503889,1489503889,20170314,6957245643416321514,None,Organic Search,Not Socially Engaged,6,...,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset
4,<NA>,1,1489503889,1489503889,20170314,6957245643416321514,None,Organic Search,Not Socially Engaged,27,...,Northern America,United States,not available in demo dataset,not available in demo dataset,not available in demo dataset,not available in demo dataset,(not set),not available in demo dataset,not available in demo dataset,not available in demo dataset


In [8]:
# Trying to save this DF
%store df_raw_flat

Stored 'df_raw_flat' (DataFrame)


In [2]:
%store -r df_raw_flat

### Feature Selection

In [3]:
# Look at missing values
print(df_raw_flat.shape)

null_counts = df_raw_flat.isnull().sum()
some_nulls = null_counts[null_counts > 0]

print(some_nulls)

(1641689, 302)
visitorId                  1641689
userId                     1641689
isSecure                   1641689
isEntrance                 1223058
isExit                     1218995
                            ...   
uniqueScreenviews          1641689
timeOnScreen               1641689
totalTransactionRevenue    1521546
sessionQualityDim          1641689
javaEnabled                1641689
Length: 230, dtype: int64


In [29]:
all_nulls = null_counts[null_counts == 1641689]
print(all_nulls)

visitorId            1641689
userId               1641689
isSecure             1641689
dataSource           1641689
productName          1641689
                      ...   
screenviews          1641689
uniqueScreenviews    1641689
timeOnScreen         1641689
sessionQualityDim    1641689
javaEnabled          1641689
Length: 158, dtype: int64


In [55]:
# Drop all columns that are completely empty
df_working = df_raw_flat.dropna(axis=1, how='all')

In [56]:
df_working.shape

(1641689, 144)

In [57]:
# Is target present in all rows?
null_counts = df_working.isnull().sum()
null_counts['action_type']

0

In [58]:
# Look at values in columns that are partially empty
some_null = null_counts[null_counts > 0]
print(some_null.index)

Index(['isEntrance', 'isExit', 'referer', 'pageTitle', 'searchKeyword',
       'searchCategory', 'transactionId', 'transactionRevenue',
       'transactionTax', 'transactionShipping', 'affiliation', 'currencyCode',
       'localTransactionRevenue', 'localTransactionTax',
       'localTransactionShipping', 'transactionCoupon', 'transactionId_1',
       'currencyCode_1', 'eventCategory', 'eventAction', 'eventLabel',
       'productSKU_1', 'v2ProductName', 'v2ProductCategory', 'productVariant',
       'productBrand', 'productRevenue', 'localProductRevenue', 'productPrice',
       'localProductPrice', 'productQuantity', 'isImpression', 'isClick',
       'productListName', 'productListPosition', 'promoId', 'promoName',
       'promoCreative', 'promoPosition', 'promoIsView', 'promoIsClick',
       'option', 'pageLoadSample', 'pageLoadTime', 'pageDownloadTime',
       'redirectionTime', 'speedMetricsSample', 'domainLookupTime',
       'serverConnectionTime', 'serverResponseTime', 'domLatencyM

In [59]:
for col in df_working:
    if col in some_null.index:
        values = df_working[col].unique()
        print(f'Unique vals in column {col}:', values, '% null:', {some_null[col] / df_working.shape[0]})

Unique vals in column isEntrance: <BooleanArray>
[<NA>, True]
Length: 2, dtype: boolean % null: {0.7449998142157254}
Unique vals in column isExit: <BooleanArray>
[<NA>, True]
Length: 2, dtype: boolean % null: {0.7425249240264142}
Unique vals in column referer: [None 'https://www.google.com.co' 'https://www.google.ca/' ...
 'http://www.bing.com/search?q=Google+store'
 'https://www.google.com.ng/search?q=google+store&oq=go&gs_l=mobile-heirloom-serp.1.0.41l3j0i131l2.3282.4341.0.12503.3.2.0.1.1.0.299.577.2-2.2.0....0...1c.1.34.mobile-heirloom-serp..0.3.632.I0NmuK0wLDg'
 'https://www.google.fr/url?sa=t&rct=j&q=&esrc=s&source=web&cd=1&ved=0ahUKEwisl9XN0sbSAhXLVxoKHejPB1sQFggaMAA&url=https%3A%2F%2Fwww.googlemerchandisestore.com%2F&usg=AFQjCNGK7lG66tWBMQkleeCMprO_xNS6qw&sig2=V1ByzvEUi5-9iFiTNbYEdQ&bvm=bv.149093890,d.d2s'] % null: {0.7590536331790004}
Unique vals in column pageTitle: ['Apparel | Google Merchandise Store'
 'Electronics | Google Merchandise Store'
 'Fun | Accessories | Google Mer

#### Reviewing the above output, I've made a list of columns to drop either because 95%+ of entries are null or because of very high cardinality in categorical columns.

In [60]:
# Reviewing above cell, drop unsalvageable columns

df_working = df_working.drop(columns=['referer', 'pageTitle', 'searchKeyword',
       'searchCategory', 'transactionId', 'transactionRevenue',
       'transactionTax', 'transactionShipping', 'affiliation', 'currencyCode',
       'localTransactionRevenue', 'localTransactionTax',
       'localTransactionShipping', 'transactionCoupon', 'transactionId_1',
       'currencyCode_1', 'eventCategory', 'eventAction', 'eventLabel',
       'productSKU_1', 'v2ProductName', 'v2ProductCategory', 'productVariant',
       'productBrand', 'productRevenue', 'localProductRevenue', 'productQuantity', 'isClick',
       'productListName', 'productListPosition', 'promoId', 'promoName',
       'promoCreative', 'promoPosition', 'promoIsView', 'promoIsClick',
       'option', 'pageLoadSample', 'pageLoadTime', 'pageDownloadTime',
       'redirectionTime', 'speedMetricsSample', 'domainLookupTime',
       'serverConnectionTime', 'serverResponseTime', 'domLatencyMetricsSample',
       'domInteractiveTime', 'domContentLoadedTime',
       'contentGroupUniqueViews1', 'contentGroupUniqueViews2',
       'contentGroupUniqueViews3', 'referralPath', 'keyword', 'adContent',
       'isTrueDirect', 'page', 'slot', 'gclId', 'adNetworkType', 'isVideoAd',
       'transactions', 'transactionRevenue_1', 'totalTransactionRevenue'])

In [61]:
df_working.shape

(1641689, 81)

In [62]:
# Identify and drop obfuscated fields
# Any such fields will be obj type since their values have been changed to a string ('not available...')
# Look at object columms and see how many of their entries are this string

obj_cols = df_working.select_dtypes(include='object')

obf_prevalence = {}
for column in obj_cols.columns:
    obf_prevalence[column] = (df_working[column] == 'not available in demo dataset').mean() * 100

In [63]:
for field in obf_prevalence:
    if obf_prevalence[field] > 0.0:
        print(field, obf_prevalence[field])

criteriaParameters 100.0
browserVersion 100.0
browserSize 100.0
operatingSystemVersion 100.0
mobileDeviceBranding 100.0
mobileDeviceModel 100.0
mobileInputSelector 100.0
mobileDeviceInfo 100.0
mobileDeviceMarketingName 100.0
flashVersion 100.0
language 100.0
screenColors 100.0
screenResolution 100.0
region 57.226916913008495
metro 57.226916913008495
city 57.226916913008495
cityId 100.0
latitude 100.0
longitude 100.0
networkLocation 100.0


In [64]:
# Drop obfuscated columns

to_drop = []
for field in obf_prevalence:
    if obf_prevalence[field] > 0.0:
        to_drop.append(field)

print(len(to_drop), to_drop)

20 ['criteriaParameters', 'browserVersion', 'browserSize', 'operatingSystemVersion', 'mobileDeviceBranding', 'mobileDeviceModel', 'mobileInputSelector', 'mobileDeviceInfo', 'mobileDeviceMarketingName', 'flashVersion', 'language', 'screenColors', 'screenResolution', 'region', 'metro', 'city', 'cityId', 'latitude', 'longitude', 'networkLocation']


In [65]:
df_working.drop(columns=to_drop, inplace=True)

In [66]:
df_working.shape

(1641689, 61)

### Examining values in remaining fields

In [67]:
df_working.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1641689 entries, 0 to 1641688
Data columns (total 61 columns):
 #   Column                          Non-Null Count    Dtype  
---  ------                          --------------    -----  
 0   visitNumber                     1641689 non-null  Int64  
 1   visitId                         1641689 non-null  Int64  
 2   visitStartTime                  1641689 non-null  Int64  
 3   date                            1641689 non-null  object 
 4   fullVisitorId                   1641689 non-null  object 
 5   channelGrouping                 1641689 non-null  object 
 6   socialEngagementType            1641689 non-null  object 
 7   hitNumber                       1641689 non-null  Int64  
 8   time                            1641689 non-null  Int64  
 9   hour                            1641689 non-null  Int64  
 10  minute                          1641689 non-null  Int64  
 11  isInteraction                   1641689 non-null  boolean
 12  

In [68]:
# Get unique values and their frequencies for remaining object cols

for col in df_working.select_dtypes(include=['object']).columns:
    print(df_working[col].value_counts())
    print()

date
20170313    63929
20170330    63692
20170307    62583
20170315    62434
20170301    60700
20170320    60121
20170302    59815
20170322    58342
20170329    57839
20170306    57750
20170308    57714
20170323    57365
20170321    57102
20170328    56366
20170327    56007
20170309    54426
20170324    53866
20170310    52948
20170314    52863
20170303    52665
20170316    51884
20170331    51358
20170304    51080
20170317    49467
20170305    46673
20170312    40313
20170318    39422
20170319    39000
20170326    38610
20170325    38584
20170311    36771
Name: count, dtype: int64

fullVisitorId
1957458976293878100    3165
4636052301667930460    2841
0824839726118485274    2597
8942866004645538967    2564
5526675926038480325    1869
                       ... 
905225556578299457        1
9304519558334289422       1
330302777359556462        1
5590489670353457051       1
2633958707935302111       1
Name: count, Length: 57888, dtype: int64

channelGrouping
Organic Search    876431
Refer

In [69]:
# After reviewing above, delete a few more columns for having no variance
to_drop = ['pagePath', 'socialEngagementType', 'hostname', 'screenDepth', 
           'socialInteractionNetworkAction', 'contentGroup4', 'contentGroup5']

df_working.drop(columns=to_drop, inplace=True)
df_working.shape

(1641689, 54)

In [70]:
# Review contents of non-object columns
for col in df_working.select_dtypes(exclude=['object']).columns:
    print(df_working[col].value_counts())
    print()

visitNumber
1      1183262
2       203087
3        79588
4        44591
5        24644
        ...   
165          1
350          1
363          1
349          1
141          1
Name: count, Length: 252, dtype: Int64

visitId
1490065543    2825
1489999542    2564
1488663704    1869
1488672213    1636
1489823400    1553
              ... 
1488643417       1
1488646169       1
1490915984       1
1490406306       1
1489080745       1
Name: count, Length: 68574, dtype: Int64

visitStartTime
1490065543    2825
1489999542    2564
1488663704    1869
1488672213    1636
1489823400    1553
              ... 
1490795856       1
1490773610       1
1490793187       1
1490251036       1
1489080745       1
Name: count, Length: 68650, dtype: Int64

hitNumber
1      416290
2      250397
3      168198
4      113377
5       84718
        ...  
463         2
483         2
494         2
448         2
481         2
Name: count, Length: 500, dtype: Int64

time
0          418651
1             143
8917         

In [72]:
# 8 more columns found that are either zero-variance, 
# redundant, or present data leakage concerns
to_drop = ['isInteraction', 'isFatal', 'visits', 
           'continent', 'subContinent', 'exitScreenName', 'hour', 'minute']

df_working.drop(columns=to_drop, inplace=True)
df_working.shape

(1641689, 46)

In [73]:
df_working.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1641689 entries, 0 to 1641688
Data columns (total 46 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   visitNumber              1641689 non-null  Int64  
 1   visitId                  1641689 non-null  Int64  
 2   visitStartTime           1641689 non-null  Int64  
 3   date                     1641689 non-null  object 
 4   fullVisitorId            1641689 non-null  object 
 5   channelGrouping          1641689 non-null  object 
 6   hitNumber                1641689 non-null  Int64  
 7   time                     1641689 non-null  Int64  
 8   isEntrance               418631 non-null   boolean
 9   isExit                   422694 non-null   boolean
 10  type                     1641689 non-null  object 
 11  pagePathLevel1           1641689 non-null  object 
 12  pagePathLevel2           1641689 non-null  object 
 13  pagePathLevel3           1641689 non-null 

* Some remaining categorical features seem important but unsure how to encode / if model will tolerate
(networkDomain country operatingSystem browser source screenName 
landingScreenName pagePathLevel1 pagePathLevel2 pagePathLevel3 pagePathLevel4)
* As a compromise to use some of the data from these I will keep values that account for the majority of rows and set the rest to 'other'

In [77]:
truncCols = ['networkDomain', 'country', 'operatingSystem', 'browser', 
             'source', 'screenName', 'landingScreenName', 'pagePathLevel1', 
             'pagePathLevel2', 'pagePathLevel3', 'pagePathLevel4']
for col in truncCols:
    print(df_working[col].value_counts().head(10))
    print()

networkDomain
(not set)              552936
unknown.unknown        191038
comcast.net             86286
verizon.net             39018
rr.com                  36454
comcastbusiness.net     34686
optonline.net           15309
hinet.net               14841
cox.net                 14187
sbcglobal.net           14077
Name: count, dtype: int64

country
United States     938478
Canada             73348
India              68098
United Kingdom     53136
Germany            29025
Taiwan             28967
Japan              26390
Australia          23628
France             23264
Spain              20191
Name: count, dtype: int64

operatingSystem
Windows          563284
Macintosh        471482
Android          236855
iOS              228533
Chrome OS         79622
Linux             53002
(not set)          6300
Windows Phone      1595
Samsung             540
BlackBerry          166
Name: count, dtype: int64

browser
Chrome               1215774
Safari                255282
Firefox                65

In [ ]:
# Continued in ingest.ipynb